# BLAS and LAPACK

We've seen a bit of dense linear algebra using numpy and scipy.  Now we're going to look under the hood.

Regardless of what language you're using, chances are if you're doing numerical linear algebra, you are able to take advantage of libraries of code which implement most common linear algebra routines and factorizations.

[Basic Linear Algebra Subprograms (BLAS)](https://www.netlib.org/blas/)

[Linear Algebra PACKage (LAPACK)](https://www.netlib.org/lapack/)

These libraries are wrapped by scipy, which exposes an interface.

## Why should you care?

First, you probably shouldn't be writing your own basic linear algebra routines if you can avoid it.
1.  It takes time to write them
2. Even if you know what you're doing, there's a chance you have a bug
3. Performance optimization is involved

It is entirely possible to do linear algebra in Python without ever worrying about the libraries under the hood.  However, maybe you are prototyping an algorithm in Python, and then want to write compiled/optimized code in C/fortran.  In this case, it is good to be able to translate what you're doing into BLAS/LAPACK routines.



## View Your Configuration

You can view what BLAS and LAPACK libraries NumPy is using

In [1]:
%pylab inline
import scipy as sp
import scipy.linalg as la

np.__config__.show()

Populating the interactive namespace from numpy and matplotlib
Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /usr/local/include
    lib directory: /usr/local/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= HASWELL MAX_THREADS=2
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep139863411681952
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing
    commands: cc
    linker: ld.bfd
    linker args: -Wl,--strip-debug, -fno-strict-aliasing
    name: gcc
    version: 10.2.1
  c++:
    commands: c++
    linker: ld.bfd
    linker args: -Wl,--strip-debug
    name: gcc
    version: 10.2.1
  cython:
    

the above show the libraries `mkl_rt`, indicating that the system is using Intel's math kernel library (MKL) - this is a library of mathematical functions (including BLAS and LAPACK) which is optimized for Intel CPUs, and is the [default for Anaconda Python](https://docs.continuum.io/mkl-optimizations/).

You can do the same for scipy:

In [2]:
sp.__config__.show()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /usr/local/include
    lib directory: /usr/local/lib
    name: openblas
    openblas configuration: USE_64BITINT=0 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= ZEN MAX_THREADS=64
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.27
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /usr/local/include
    lib directory: /usr/local/lib
    name: openblas
    openblas configuration: USE_64BITINT=0 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= ZEN MAX_THREADS=64
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.27
  pybind11:
    detection method: config-tool
    include directory: unknown
    name: pybind11
    version: 2.12.0
Compilers:
  c:
    commands: cc
    linker: ld.bfd
    name: gcc
    version: 10.2.1
  c++:
    commands: c++

## BLAS Routines

[scipy BLAS interface](https://docs.scipy.org/doc/scipy/reference/linalg.blas.html)

BLAS implements *basic* linear algebra routines like dot product, matrix-vector product, and matrix-matrix product as well as triangular solves.

It is written in Fortran, so will be easiest to use if you set the flag `order='F'` when constructing arrays

In [3]:
from scipy.linalg import blas

n = 4096
A = np.array(np.random.randn(n,n), order='F')
B = np.array(np.random.randn(n,n), order='F')


C_np = A @ B # numpy  C = A * B
C_blas = blas.dgemm(1.0, A, B) # BLAS C = A * B
np.linalg.norm(C_np - C_blas)

0.0

In [4]:
%timeit C_np = np.matmul(A,B) # numpy

6.96 s ± 914 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit C_blas = blas.dgemm(1.0, A, B) # blas

4.79 s ± 627 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### BLAS Naming Conventions

BLAS functions have short names (like `dgemm`) that look a little cryptic at first.  There is a pattern to the names though.  See [here](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-fortran/top/blas-and-sparse-blas-routines/blas-routines/naming-conventions-for-blas-routines.html) for reference.

```
blas.<character><name><mod>
```

The `<character>` field can be
1. `s`: single precision float
2. `d`: double precision float
3. `c`: single precision complex float
4. `z`: double precision complex float

The `<name>` field indicate either the operation type, or matrix type.

The `<mod>` field indicates additional details about the operation.

For example, `dgemm = d + ge + mm` has `d` as the `<character>`, `ge` for `<name>` (general matrix), and `mm` for `<mod>` (matrix multiplication).

### BLAS Levels

BLAS is split up into 3 conceptual levels
1. Level 1 - vector-vector operations
2. Level 2 - matrix-vector operations
3. Level 3 - matrix-matrix operations

Level 3 operations are most efficient, since they can take the most advantage of memory performance optimizations e.g. minimizing cache misses.

#### FLOPS

FLOPS are Floating-Point Operations Per Second, and are one way to measure the power of numerical code.  Floating point operations are counted as the number of `+, *, /, -` applied to floating point numbers.
* A human is capable of <1 flop
* Your CPU is probably capable of several Giga-flops ($10^9$)
* A high end GPU will be measured in Tera-flops ($10^{12}$)
* Super computers are mostly O(100) Peta-flops ($10^{17}$).  Some are close to an Exa-flop ($10^{18}$)

#### BLAS Level 1

Some vector-vector operations (insert the appropriate `<character>`)
1. [`axpy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.daxpy.html) ($a x + y$)
2. [`dot`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.ddot.html) (dot product $x^H x$)
3. [`nrm2`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dnrm2.html) ($\|x\|_2$)

See [the SciPy BLAS reference](https://docs.scipy.org/doc/scipy/reference/linalg.blas.html#blas-level-1-functions)

In [6]:
n = 1000
x = np.random.rand(n)
y = np.random.rand(n)

z = blas.daxpy(x, y, a=1.0)
xx = blas.ddot(x, x)
x2 = blas.dnrm2(x)
np.sqrt(xx) - x2

0.0

In [7]:
import time

In [8]:
# measure FLOPs of fdot
n = 10_000_000
dtype = np.float32
x = np.array(np.random.randn(n), dtype=dtype)
y = np.array(np.random.randn(n), dtype=dtype)
print("measuring BLAS Level 1 flops using {}".format(dtype))
t0 = time.time()
xy = blas.sdot(x,y)
t1 = time.time()
print("time elapsed = {} sec.".format(t1 - t0))
flops = (2*n) / (t1 - t0) # 2N FLOP for dot - one multiplication and one addition per entry
print("{:e} FLOPS".format(flops))

measuring BLAS Level 1 flops using <class 'numpy.float32'>
time elapsed = 0.012787342071533203 sec.
1.564047e+09 FLOPS


This is consistent with a CPU running at ~3 GHz

#### BLAS Level 2

Some matrix-vector operations (insert appropriate `<character>`)

1. [`gemv`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dgemv.html#scipy.linalg.blas.dgemv) $\alpha A x$
2. [`trsv`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dtrsv.html#scipy.linalg.blas.dtrsv) $L^{-1} x$ (triangular solve)
3. [`trmv`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.strmv.html#scipy.linalg.blas.strmv) $L x$ (triangular matrix-vector product)

See [the SciPy BLAS reference](https://docs.scipy.org/doc/scipy/reference/linalg.blas.html#blas-level-2-functions)

In [9]:
# measure FLOPs of gemv
n = 4096
dtype = np.float32
A = np.array(np.random.randn(n, n), dtype=dtype)
x = np.array(np.random.randn(n), dtype=dtype)
print("measuring BLAS Level 2 flops using {}".format(dtype))
t0 = time.time()
y = blas.sgemv(1.0, A, x)
t1 = time.time()
print("time elapsed = {} sec.".format(t1 - t0))
flops = (2 * n**2) / (t1 - t0) # 2 n**2 FLOP - one multiplication and one addition per entry of A
print("{:e} FLOPS".format(flops))

measuring BLAS Level 2 flops using <class 'numpy.float32'>
time elapsed = 0.22038793563842773 sec.
1.522517e+08 FLOPS


#### BLAS Level 3

Some matrix-matrix operations (insert appropriate `<character>`)

1. [`gemm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dgemm.html#scipy.linalg.blas.dgemm) $\alpha AB$
2. [`syrk`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dsyrk.html#scipy.linalg.blas.dsyrk) $\alpha A A^T$
3. [`trmm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.blas.dtrmm.html#scipy.linalg.blas.dtrmm) $\alpha L_1 L_2$ (triangular multiplication)

See the [SciPy BLAS reference](https://docs.scipy.org/doc/scipy/reference/linalg.blas.html#blas-level-3-functions)

In [10]:
# measure FLOPs of gemm
n = 4096
dtype = np.float32
A = np.array(np.random.randn(n, n), dtype=dtype)
B = np.array(np.random.randn(n, n), dtype=dtype)
print("measuring BLAS Level 3 flops using {}".format(dtype))
t0 = time.time()
C = blas.sgemm(1.0, A, B)
t1 = time.time()
print("time elapsed = {} sec.".format(t1 - t0))
flops = (2 * n**3) / (t1 - t0) # 2 n**3 FLOP - n multiplications and additions per entry of C
print("{:e} FLOPS".format(flops))

measuring BLAS Level 3 flops using <class 'numpy.float32'>
time elapsed = 2.4508888721466064 sec.
5.607719e+10 FLOPS


From our experminets, we see that the BLAS level 3 operation gives us the most FLOPS.

## LAPACK Routines

[scipy LAPACK interface](https://docs.scipy.org/doc/scipy/reference/linalg.lapack.html)

In [11]:
from scipy.linalg import lapack

LAPACK is a library of linear algebra routines that go beyond basic operations.  These include routines for various factorizations and eigenvalue and singular value decompositions.

Again, the names are a bit cryptic, and it is worth searching online (and reading documentation) to figure out how to call the right functions.

### Example: Obtain a orthonormal basis for columns of a matrix

In a variety of situations it is convenient to be able to obtain an orthonormal basis for columns of a matrix `A` (e.g. subspace iteration, randomized numerical linear algebra).  One way to do this is to use the QR decomposition:

In [12]:
m = 1000
n = 100
A = np.array(np.random.randn(m, n), order='F')

def get_Q_qr(A):
    Q, R = la.qr(A, mode='economic')
    return Q

Q = get_Q_qr(A)

print(Q.shape)
print(np.linalg.norm(Q.T @ Q - np.eye(n))) # measure how close Q is to orthogonal

(1000, 100)
4.225444383430174e-15


We can get this using LAPACK routines as well:

In [13]:
def get_Q_lapack(A):
    qr, tau, work, info = lapack.dgeqrf(A)
    Q, work, info = lapack.dorgqr(qr, tau)
    return Q

A = np.array(np.random.randn(m, n), order='F')
Q = get_Q_lapack(A)

print(Q.shape)
print(np.linalg.norm(Q.T @ Q - np.eye(n))) # measure how close Q is to orthogonal

(1000, 100)
4.1695759733668806e-15


You can also do many LAPACK (and BLAS) routines in-place, meaning you can overwrite the matrix.  This saves time used for memory allocation

In [14]:
def get_Q_inplace(A):
    m, n = A.shape
    lwork = max(3*n, 1)
    qr, tau, work, info = lapack.dgeqrf(A, lwork, 1) # overwrite A = True
    Q, work, info = lapack.dorgqr(qr, tau, lwork, 1) # overwrite qr = True
    return Q

A = np.array(np.random.randn(m, n), order='F')
Q = get_Q_inplace(A)

print(Q.shape)
print(np.linalg.norm(Q.T @ Q - np.eye(n))) # measure how close Q is to orthogonal
print(np.linalg.norm(A - Q)) # A now contains Q

(1000, 100)
4.349967906377366e-15
0.0


In [15]:
m = 4096
n = 128

for get_Q in (get_Q_qr, get_Q_lapack, get_Q_inplace):

    print("timing {}".format(get_Q))
    A = np.array(np.random.randn(m, n), order='F')
    t0 = time.time()
    Q = get_Q(A)
    t1 = time.time()
    print("  time elapsed = {} sec.".format(t1 - t0))

timing <function get_Q_qr at 0x7e95cd1327a0>
  time elapsed = 0.08739638328552246 sec.
timing <function get_Q_lapack at 0x7e95cd133880>
  time elapsed = 0.06744122505187988 sec.
timing <function get_Q_inplace at 0x7e95cd133c70>
  time elapsed = 0.051262617111206055 sec.


As we see, the in-place version is fastest.

## Exercise

One way to get the top `k`-dimensional eigenspace (eigenpace with k-largest eigenvalues) is using a subspace version of the power method.  Here is some pseudocode:
```

# get top k-dimensional eigenspace of symmetric matrix A
m, n = A.shape # m should equal n
Q = random n x k matrix
Q, R = qr(Q) # orthogonalize
for some number of iterations:
    Q = A @ Q
    Q, R = qr(Q) # re-othogonalize
    
return Q
```

Implement a function that realizes this algorithm.  Use BLAS to implement matrix-matrix multiplication, and use LAPACK to do the orthogonalization of Q.

Compare this to the span of the top-k eigenvectors of `A` obtained via `eigh`.

In [18]:
import numpy as np
def subspace_iter(A, k, num_iter=10):
  m, n = A.shape
  Q = np.array(np.random.randn(n, k), order='F')

  #orthogonalize
  qr, tau, work, info = lapack.dgeqrf(Q)
  Q, work, info = lapack.dorgqr(qr, tau)

  for i in range(num_iter):
    Q = blas.dgemm(1.0, A, Q)
    #re-orthogonalize
    qr, tau, work, info = lapack.dgeqrf(Q)
    Q, work, info = lapack.dorgqr(qr, tau)

  return Q


In [19]:
import numpy as np
n = 1000
k = 10
A = np.random.randn(n, n)
A = A + A.T # make symmetric

Q_si = subspace_iter(A, k, num_iter=100)

from scipy.linalg import eigh

evals, evecs = eigh(A)
Q_eigh = evecs[:, -k:]

# measure the distance between the subspaces
from numpy.linalg import svd
U, S, VT = svd(Q_eigh.T @ Q_si)
print("distance between subspaces = ", np.sqrt(1 - S[-1]**2))


distance between subspaces =  0.999999890497413


In [16]:
## Your Code here


## Exercise

Implement a version of `solve` for a square matrix `A` which uses LAPACK for the LU decomposition, and BLAS for the triangular solves.  You may want to look at [dgetrf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lapack.dgetrf.html#scipy.linalg.lapack.dgetrf) (the `lu` return object has L in the lower triangular part, and U in the upper triangular part)

In [20]:
import numpy as np
def solve_lu(A, b):
  lu, piv, info = lapack.dgetrf(A)
  x = lapack.dgetrs(lu, piv, b)[0]
  return x

n = 1000
A = np.array(np.random.randn(n, n), order='F')
b = np.random.randn(n)

x_lu = solve_lu(A.copy(), b.copy()) # make copies since dgetrf overwrites A
x_np = np.linalg.solve(A, b)

np.linalg.norm(x_lu - x_np)


0.0

In [17]:
## Your code here
